In [1]:
# Library for opening url and creating requests
import urllib.request
import html_table_parser

# pretty-print python data structures
from pprint import pprint

# for parsing all the tables present on the website
# from html_table_parser.parser import HTMLTableParser
from html_table_parser import HTMLTableParser


# for converting the parsed data in a pandas dataframe
import pandas as pd

ImportError: cannot import name 'HTMLTableParser' from 'html_table_parser' (C:\Users\Pranav Jaju\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\html_table_parser\__init__.py)

In [2]:
# Opens a website and read its binary contents (HTTP Response Body)
def url_get_contents(url):

    # Opens a website and read its binary contents (HTTP Response Body)
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'}
    #making request to the website
    req = urllib.request.Request(url=url,headers=headers)
    f = urllib.request.urlopen(req)

    #reading contents of the website
    return f.read()

In [3]:
# Erling Haaland All Stats for Manchester City
Link = 'https://www.transfermarkt.co.in/erling-haaland/leistungsdatendetails/spieler/418560/plus/1?saison=&verein=281&liga=&wettbewerb=&pos=&trainer_id=' 

In [4]:
# defining the html contents of a URL.
xhtml = url_get_contents(Link).decode('utf-8')

# Defining the HTMLTableParser object
p = HTMLTableParser()

# feeding the html contents in the HTMLTableParser object
p.feed(xhtml)

In [5]:
def header_convert(df):
    # Combine the top 2 rows into a single header row
    new_header = df.iloc[0].fillna('')
    df = df[1:]  # Exclude the first row
    
    # Assign the combined row as the new header
    df.columns = new_header
    df.reset_index(drop=True, inplace=True)

    # Rename columns directly
    df.columns = ['Matchday','Date','HT_logo','HT','AT_logo','AT','Result','Position','Goals','Assists','OG','YC','2YC','RC','Sub_in','Sub_out','MP']
    
    # Drop unnecessary columns
    df.drop(columns=['HT_logo', 'AT_logo'], inplace=True)
    
    # Remove rows where 'Date' is NaN and convert 'Date' column to datetime
    df = df[df['Date'].notnull()].copy()
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    
    # Fill NaNs with 0 and convert specific columns to numeric
    df.fillna(0, inplace=True)
    numeric_cols = ['Goals', 'Assists', 'OG']
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')
    
    return df


In [6]:
# Returning a view versus a copy
pd.options.mode.copy_on_write = True

In [7]:
## All Table Details for reference
# Table 1 > Summary Table
# Table 2 > UEFA Champions League
# Table 3 > UEFA Super Cup
# Table 4 > FIFA Club World Cup 
# Table 5 > Premier League
# Table 6 > Community Shield
# Table 7 > FA Cup
# Table 8 > EFL Cup

In [8]:
import pandas as pd

def process_competition_tables(tables, indices, drop_suffix=True):
    def process_table(table, drop_suffix):
        df = header_convert(pd.DataFrame(table))
        if drop_suffix:
            df['HT'] = df['HT'].str[:-5] 
            df['AT'] = df['AT'].str[:-5]
        return df

    processed_frames = [process_table(tables[i], drop_suffix=(i == indices['epl'])) for i in indices.values()]

    combined_result = pd.concat(processed_frames).sort_values('Date')
    return combined_result

# Dictionary to store the table indices
table_indices = {
    'ucl': 2,
    'usc': 3,
    'epl': 5,
    'cs': 6,
    'fa': 7,
    'efl': 8
}

# Assuming 'p.tables' contains all the necessary tables
result = process_competition_tables(p.tables, table_indices)


In [ ]:
# # All Competition Tables
# uefa_c_table = pd.DataFrame(p.tables[2])
# ucl = header_convert(uefa_c_table)

# uefa_super_cup_table = pd.DataFrame(p.tables[3])
# usc = header_convert(uefa_super_cup_table)

# fifa_club_worldcup_table = pd.DataFrame(p.tables[4])
# fcw = header_convert(fifa_club_worldcup_table)

# premier_league_table = pd.DataFrame(p.tables[5])
# epl = header_convert(premier_league_table)
# epl['HT'] = epl['HT'].str[:-5] 
# epl['AT'] = epl['AT'].str[:-5]

# community_shield_table = pd.DataFrame(p.tables[6])
# cs = header_convert(community_shield_table)

# fa_cup_table = pd.DataFrame(p.tables[7])
# fa = header_convert(fa_cup_table)

# efl_cup_table = pd.DataFrame(p.tables[8])
# efl = header_convert(efl_cup_table)

# # Concatinating Data across all competition
# frames = [ucl,usc,epl,cs,fa,efl]

# result = pd.concat(frames)
# result = result.sort_values('Date')

In [12]:
matches_played = result[result['Position'] == "CF"]

In [13]:
matches_played

,Matchday,Date,HT,AT,Result,Position,Goals,Assists,OG,YC,2YC,RC,Sub_in,Sub_out,MP
0,Final,2022-07-30,Liverpool,Man City,3:1,CF,NaN,NaN,NaN,,,,,,90'
0,1,2022-08-07,West Ham,Man City,0:2,CF,2.0,NaN,NaN,,,,,78',78'
1,2,2022-08-13,Man City,Bournemouth,4:0,CF,NaN,1.0,NaN,,,,,74',74'
2,3,2022-08-21,Newcastle,Man City,3:3,CF,1.0,NaN,NaN,,,,,,90'
3,4,2022-08-27,Man City,Crystal Palace,4:2,CF,3.0,NaN,NaN,,,,,84',84'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12,Final,2024-05-25,Man City,Man Utd,1:2,CF,NaN,NaN,NaN,,,,,,90'
4,Final,2024-08-10,Man Utd,Man City,7:8 on pens,CF,NaN,NaN,NaN,,,,,,90'
78,1,2024-08-18,Chelsea,Man City,0:2,CF,1.0,NaN,NaN,66',,,,,90'
79,2,2024-08-24,Man City,Ipswich,4:1,CF,3.0,NaN,NaN,,,,,90',90'


In [ ]:
# matches_played.to_csv("erling_haaland_101_matches.csv",index=False)